In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from itertools import product
from datetime import date
from path import Path
import pandas as pd
import random

import os
os.environ['NUMEXPR_MAX_THREADS'] = '32'

from mlbt.run_bt import run_bt
from mlbt.feature_eng import define_feature_configs
from mlbt.load_data import LENGTH_RANKING


DATA_DIR = Path("~/Dropbox/algotrading/data").expanduser()
s_csv = pd.read_csv(DATA_DIR / "symbols.csv", index_col="iqsymbol")

# Code to execute the pipeline inside a multiprocessing.Pool
# This creates the payloads that "run_frontend_payloads.ipynb" can use to generate the
# frontend payloads (.json loaded by the frontend)
ranking = ["equity_index", ""]


symbol_groups = list(
    {
        "agriculture": "Agriculture",
        "currency": "Currency",
        "energy": "Energy",
        "equity_index": "Equity Index",
        "interest_rate": "Interest Rate",
        "metals": "Metals",
    }.keys()
)

features = define_feature_configs()
print(len(features))
c_s = (len(features) // 32) + 1
data_sim = {
    "start_date": [date(2000, 1, 1)],
    "end_date": [date(2021, 1, 1)],
    "symbol_groups": [symbol_groups],
#     "symbol_groups": [[x] for x in symbol_groups],
#     "symbol_groups": [['equity_index'], ['interest_rate']],
#     "symbols": [['@NQ#C']],
#     "symbols": [[x] for x in LENGTH_RANKING],
    "bar_type": ["dollar"],
    "binarize": ["fixed_horizon"],
    "binarize_params": [10, 25, 50, 100],
#     "binarize_params": [10, 25, 100],
#     "binarize_params": [50],
    "alpha": ["none"],
#     "features": [features[i:i+c_s] for i in range(0, len(features), c_s)],
    "features": [features],
#     "classifier": ["random_forest", "xgboost", "lgbm", "knn", "dummy"],
    "classifier": ["random_forest", "xgboost", "dummy"],
#     "classifier": ["tpot"],
#     "classifier": ["lgbm"],
    "num_threads": [32], # how many parallel hyperopti 
    "n_jobs": [32], # how many parallel infer
    "func": [run_bt],
#     "feature_calc_only": [True],
#     "feature_imp_only": [True],
    "check_completed": [False],
#     "optimize_hypers": [False],
    "reuse_hypers": [False],
#     "load_from_disk": [False],
}

65


In [5]:
datas = list(dict(zip(data_sim.keys(), x)) for x in product(*data_sim.values()))
# datas
# for data in datas:
#     random.shuffle(data['features'])


In [6]:
len(datas)

12

In [7]:
from mlbt.multiprocess import process_jobs

In [8]:
num_threads = max(1, min(32, len(datas)) - 0)
num_threads
      

12

In [9]:
# process_jobs(datas, num_threads=num_threads)

In [ ]:
for data in datas:
    run_bt(**data)

2020-02-12 10:06:17,285 config: {'start_date': datetime.date(2000, 1, 1), 'end_date': datetime.date(2021, 1, 1), 'vol_estimate': 100, 'downsampling': 'cusum', 'symbols': None, 'symbol_groups': ['agriculture', 'currency', 'energy', 'equity_index', 'interest_rate', 'metals'], 'features': [{'name': 'log_ret'}, {'name': 'log_ret', 'symbol': {'name': 'time_bars', 'size': 60, 'symbol': 'VIX.XO'}}, {'name': 'ffd', 'd': 0.3}, {'name': 'log_ret', 'periods': 10}, {'name': 'roll', 'window': 10}, {'name': 'rollimp', 'window': 10}, {'name': 'amihud', 'window': 10}, {'name': 'kyle', 'window': 10}, {'name': 'volratio', 'com': 10}, {'name': 'lag', 'lag': 10, 'symbol': {'name': 'volratio', 'com': 10}}, {'name': 'stdev', 'window': 10, 'symbol': {'name': 'log_ret', 'symbol': {'name': 'time_bars', 'size': 60, 'symbol': 'VIX.XO'}}}, {'name': 'stdev', 'window': 10, 'symbol': {'name': 'log_ret'}}, {'name': 'stdev', 'window': 10, 'symbol': {'name': 'stdev', 'window': 10, 'symbol': {'name': 'log_ret'}}}, {'nam

2020-02-12 10:06:17,761 @CC#C: Have 61619 bars and 29066 binarized events
2020-02-12 10:06:17,774 @CT#C: Have 61020 bars and 28689 binarized events
2020-02-12 10:06:17,788 @OJ#C: Have 72862 bars and 34193 binarized events
2020-02-12 10:06:17,803 @KC#C: Have 86403 bars and 41560 binarized events
2020-02-12 10:06:17,818 @SB#C: Have 87741 bars and 34203 binarized events
2020-02-12 10:06:17,830 GAS#C: Have 27979 bars and 13228 binarized events
2020-02-12 10:06:17,843 LF#C: Have 63915 bars and 30955 binarized events
2020-02-12 10:06:17,856 LG#C: Have 48041 bars and 22659 binarized events
2020-02-12 10:06:17,870 QCL#C: Have 63981 bars and 30456 binarized events
2020-02-12 10:06:17,884 QHO#C: Have 83962 bars and 8213 binarized events
2020-02-12 10:06:17,899 QNG#C: Have 127788 bars and 30813 binarized events
2020-02-12 10:06:17,912 QPA#C: Have 32928 bars and 15905 binarized events
2020-02-12 10:06:17,924 QPL#C: Have 50590 bars and 24800 binarized events
2020-02-12 10:06:17,942 @QG#C: Have 1860

2020-02-12 10:06:18,278 Got (27209,) shape for feature: {"name": "stdev", "symbol": "{\"name\": \"log_ret\", \"symbol\": \"{\\\"name\\\": \\\"time_bars\\\", \\\"size\\\": 60, \\\"symbol\\\": \\\"VIX.XO\\\"}\"}", "window": 250}
2020-02-12 10:06:18,285 Got (84868,) shape for feature: {"name": "stdev", "symbol": "{\"name\": \"log_ret\"}", "window": 250}
2020-02-12 10:06:18,291 Got (84868,) shape for feature: {"name": "stdev", "symbol": "{\"name\": \"stdev\", \"symbol\": \"{\\\"name\\\": \\\"log_ret\\\"}\", \"window\": 250}", "window": 250}
2020-02-12 10:06:18,296 Got (84868,) shape for feature: {"name": "stdev", "symbol": "{\"d\": 0.3, \"name\": \"ffd\"}", "window": 250}
2020-02-12 10:06:18,301 Got (27209,) shape for feature: {"lag": 10, "name": "auto", "symbol": "{\"name\": \"log_ret\", \"symbol\": \"{\\\"name\\\": \\\"time_bars\\\", \\\"size\\\": 60, \\\"symbol\\\": \\\"VIX.XO\\\"}\"}", "window": 250}
2020-02-12 10:06:18,308 Got (84868,) shape for feature: {"lag": 10, "name": "auto", "s

2020-02-12 10:06:18,613 Got (27209,) shape for feature: {"lag": 25, "name": "auto", "symbol": "{\"name\": \"log_ret\", \"symbol\": \"{\\\"name\\\": \\\"time_bars\\\", \\\"size\\\": 60, \\\"symbol\\\": \\\"VIX.XO\\\"}\"}", "window": 50}
2020-02-12 10:06:18,619 Got (92410,) shape for feature: {"lag": 25, "name": "auto", "symbol": "{\"com\": 50, \"name\": \"volratio\"}", "window": 50}
2020-02-12 10:06:18,624 Got (92410,) shape for feature: {"lag": 25, "name": "auto", "symbol": "{\"name\": \"stdev\", \"symbol\": \"{\\\"com\\\": 50, \\\"name\\\": \\\"volratio\\\"}\", \"window\": 50}", "window": 50}
2020-02-12 10:06:18,630 Got (92410,) shape for feature: {"name": "log_ret", "periods": 250}
2020-02-12 10:06:18,635 Got (92410,) shape for feature: {"name": "roll", "window": 250}
2020-02-12 10:06:18,640 Got (92410,) shape for feature: {"name": "rollimp", "window": 250}
2020-02-12 10:06:18,646 Got (92410,) shape for feature: {"name": "amihud", "window": 250}
2020-02-12 10:06:18,651 Got (92410,) s

2020-02-12 10:06:18,966 Got (86716,) shape for feature: {"name": "stdev", "symbol": "{\"name\": \"log_ret\"}", "window": 50}
2020-02-12 10:06:18,972 Got (86716,) shape for feature: {"name": "stdev", "symbol": "{\"name\": \"stdev\", \"symbol\": \"{\\\"name\\\": \\\"log_ret\\\"}\", \"window\": 50}", "window": 50}
2020-02-12 10:06:18,978 Got (86716,) shape for feature: {"name": "stdev", "symbol": "{\"d\": 0.3, \"name\": \"ffd\"}", "window": 50}
2020-02-12 10:06:18,982 Got (27209,) shape for feature: {"lag": 10, "name": "auto", "symbol": "{\"name\": \"log_ret\", \"symbol\": \"{\\\"name\\\": \\\"time_bars\\\", \\\"size\\\": 60, \\\"symbol\\\": \\\"VIX.XO\\\"}\"}", "window": 50}
2020-02-12 10:06:18,989 Got (86716,) shape for feature: {"lag": 10, "name": "auto", "symbol": "{\"com\": 50, \"name\": \"volratio\"}", "window": 50}
2020-02-12 10:06:18,994 Got (86716,) shape for feature: {"lag": 10, "name": "auto", "symbol": "{\"name\": \"stdev\", \"symbol\": \"{\\\"com\\\": 50, \\\"name\\\": \\\"vo

...

2020-02-12 10:06:34,622 Got (67132,) shape for feature: {"lag": 10, "name": "lag", "symbol": "{\"com\": 10, \"name\": \"volratio\"}"}
2020-02-12 10:06:34,626 Got (27209,) shape for feature: {"name": "stdev", "symbol": "{\"name\": \"log_ret\", \"symbol\": \"{\\\"name\\\": \\\"time_bars\\\", \\\"size\\\": 60, \\\"symbol\\\": \\\"VIX.XO\\\"}\"}", "window": 10}
2020-02-12 10:06:34,633 Got (67132,) shape for feature: {"name": "stdev", "symbol": "{\"name\": \"log_ret\"}", "window": 10}
2020-02-12 10:06:34,639 Got (67132,) shape for feature: {"name": "stdev", "symbol": "{\"name\": \"stdev\", \"symbol\": \"{\\\"name\\\": \\\"log_ret\\\"}\", \"window\": 10}", "window": 10}
2020-02-12 10:06:34,644 Got (67132,) shape for feature: {"name": "stdev", "symbol": "{\"d\": 0.3, \"name\": \"ffd\"}", "window": 10}
2020-02-12 10:06:34,649 Got (67132,) shape for feature: {"name": "log_ret", "periods": 25}
2020-02-12 10:06:34,654 Got (67132,) shape for feature: {"name": "roll", "window": 25}
2020-02-12 10:06

2020-02-12 10:06:34,907 Got (67132,) shape for feature: {"lag": 50, "name": "auto", "symbol": "{\"com\": 250, \"name\": \"volratio\"}", "window": 250}
2020-02-12 10:06:34,912 Got (67132,) shape for feature: {"lag": 50, "name": "auto", "symbol": "{\"name\": \"stdev\", \"symbol\": \"{\\\"com\\\": 250, \\\"name\\\": \\\"volratio\\\"}\", \"window\": 250}", "window": 250}
2020-02-12 10:06:34,932 Joined 65 features into (67132, 65) shape
2020-02-12 10:06:34,933 QSI#C: Feature engineering for 65 features
2020-02-12 10:06:34,942 Got (59640,) shape for feature: {"name": "log_ret"}
2020-02-12 10:06:34,947 Got (27209,) shape for feature: {"name": "log_ret", "symbol": "{\"name\": \"time_bars\", \"size\": 60, \"symbol\": \"VIX.XO\"}"}
2020-02-12 10:06:34,953 Got (59640,) shape for feature: {"d": 0.3, "name": "ffd"}
2020-02-12 10:06:34,958 Got (59640,) shape for feature: {"name": "log_ret", "periods": 10}
2020-02-12 10:06:34,963 Got (59640,) shape for feature: {"name": "roll", "window": 10}
2020-02-

2020-02-12 10:06:35,232 Got (59640,) shape for feature: {"lag": 10, "name": "auto", "symbol": "{\"com\": 250, \"name\": \"volratio\"}", "window": 250}
2020-02-12 10:06:35,237 Got (59640,) shape for feature: {"lag": 10, "name": "auto", "symbol": "{\"name\": \"stdev\", \"symbol\": \"{\\\"com\\\": 250, \\\"name\\\": \\\"volratio\\\"}\", \"window\": 250}", "window": 250}
2020-02-12 10:06:35,242 Got (27209,) shape for feature: {"lag": 25, "name": "auto", "symbol": "{\"name\": \"log_ret\", \"symbol\": \"{\\\"name\\\": \\\"time_bars\\\", \\\"size\\\": 60, \\\"symbol\\\": \\\"VIX.XO\\\"}\"}", "window": 250}
2020-02-12 10:06:35,249 Got (59640,) shape for feature: {"lag": 25, "name": "auto", "symbol": "{\"com\": 250, \"name\": \"volratio\"}", "window": 250}
2020-02-12 10:06:35,254 Got (59640,) shape for feature: {"lag": 25, "name": "auto", "symbol": "{\"name\": \"stdev\", \"symbol\": \"{\\\"com\\\": 250, \\\"name\\\": \\\"volratio\\\"}\", \"window\": 250}", "window": 250}
2020-02-12 10:06:35,259

2020-02-12 10:06:41,130 Dropped label 0.0 0.008348317492936038
2020-02-12 10:06:41,339 bars (33182, 9), events (15582, 2), feats (33182, 65), bins (15442, 3), X_all (14262, 65), X_train (7131, 65)
2020-02-12 10:06:41,345 @O#C: Get bins and feature imps
2020-02-12 10:06:41,359 Dropped label 0.0 0.01614415589710139
2020-02-12 10:06:41,594 bars (56471, 9), events (24539, 2), feats (56471, 65), bins (24133, 3), X_all (23108, 65), X_train (11554, 65)
2020-02-12 10:06:41,600 @SM#C: Get bins and feature imps
2020-02-12 10:06:41,613 Dropped label 0.0 0.011192099694333412
2020-02-12 10:06:41,835 bars (44130, 9), events (21275, 2), feats (44130, 65), bins (21027, 3), X_all (19744, 65), X_train (9872, 65)
2020-02-12 10:06:41,842 @S#C: Get bins and feature imps
2020-02-12 10:06:41,858 Dropped label 0.0 0.010452860782951124
2020-02-12 10:06:42,119 bars (72162, 9), events (34546, 2), feats (72162, 65), bins (34175, 3), X_all (32980, 65), X_train (16490, 65)
2020-02-12 10:06:42,125 @W#C: Get bins and

2020-02-12 10:06:47,466 @TU#C [(7175, 5), (7175, 65), (7175,), (7176, 5), (7176, 65), (7176,)]
2020-02-12 10:06:47,466 @UB#C [(7131, 5), (7131, 65), (7131,), (7131, 5), (7131, 65), (7131,)]
2020-02-12 10:06:47,467 @O#C [(11554, 5), (11554, 65), (11554,), (11554, 5), (11554, 65), (11554,)]
2020-02-12 10:06:47,467 @SM#C [(9872, 5), (9872, 65), (9872,), (9872, 5), (9872, 65), (9872,)]
2020-02-12 10:06:47,467 @S#C [(16490, 5), (16490, 65), (16490,), (16490, 5), (16490, 65), (16490,)]
2020-02-12 10:06:47,468 @W#C [(24331, 5), (24331, 65), (24331,), (24332, 5), (24332, 65), (24332,)]
2020-02-12 10:06:47,468 @MME#C [(4720, 5), (4720, 65), (4720,), (4721, 5), (4721, 65), (4721,)]
2020-02-12 10:06:47,468 @CC#C [(13805, 5), (13805, 65), (13805,), (13805, 5), (13805, 65), (13805,)]
2020-02-12 10:06:47,468 @CT#C [(13663, 5), (13663, 65), (13663,), (13663, 5), (13663, 65), (13663,)]
2020-02-12 10:06:47,469 @OJ#C [(16427, 5), (16427, 65), (16427,), (16427, 5), (16427, 65), (16427,)]
2020-02-12 10:06

In [ ]:
%debug

In [ ]:
1

[autoreload of tornado.util failed: Traceback (most recent call last):
  File "c:\users\doda\anaconda3\envs\fincl\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "c:\users\doda\anaconda3\envs\fincl\lib\site-packages\IPython\extensions\autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "c:\users\doda\anaconda3\envs\fincl\lib\site-packages\IPython\extensions\autoreload.py", line 347, in update_generic
    update(a, b)
  File "c:\users\doda\anaconda3\envs\fincl\lib\site-packages\IPython\extensions\autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
  File "c:\users\doda\anaconda3\envs\fincl\lib\site-packages\IPython\extensions\autoreload.py", line 347, in update_generic
    update(a, b)
  File "c:\users\doda\anaconda3\envs\fincl\lib\site-packages\IPython\extensions\autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_

1

In [ ]:
print '*' * 100000000